In [1]:
import os
import pickle
import numpy as np
import torch
import torch.nn
import torch.autograd

In [2]:
#data = pickle.load(open( "/home/yuxuan/gpnn/tmp/hico/processed/hico_data_background_49/HICO_train2015_00029234.p", "rb" ))
data = pickle.load(open( "/home/yuxuan/project/gpnn/tmp/hico/processed/hico_data_background_49/HICO_test2015_00000001.p", "rb" ))
edge_features = np.load("/home/yuxuan/project/gpnn/tmp/hico/processed/hico_data_background_49/HICO_test2015_00000001_edge_features.npy")
node_features = np.load("/home/yuxuan/project/gpnn/tmp/hico/processed/hico_data_background_49/HICO_test2015_00000001_node_features.npy")

UnicodeDecodeError: 'ascii' codec can't decode byte 0xf0 in position 22: ordinal not in range(128)

In [ ]:
# Link function
class LinkFunction(torch.nn.Module):
    def __init__(self, link_def, args):
        super(LinkFunction, self).__init__()
        self.l_definition = ''
        self.l_function = None
        self.learn_args = torch.nn.ParameterList([])
        self.learn_modules = torch.nn.ModuleList([])
        self.__set_link(link_def, args)

    def forward(self, edge_features):
        return self.l_function(edge_features)

    def __set_link(self, link_def, args):
        self.l_definition = link_def.lower()    # set input to lowercase
        self.args = args

        self.l_function = {
            'graphconv':        self.l_graph_conv,
            'graphconvlstm':    self.l_graph_conv_lstm,
        }.get(self.l_definition, None)

        # self.l_function = self.l_graph_conv

        if self.l_function is None:
            print('WARNING!: Update Function has not been set correctly\n\tIncorrect definition ' + link_def)
            quit()

        init_parameters = {
            'graphconv':        self.init_graph_conv,
            'graphconvlstm':    self.init_graph_conv_lstm,
        }.get(self.l_definition, lambda x: (torch.nn.ParameterList([]), torch.nn.ModuleList([]), {}))

        # init_parameters = self.init_graph_conv()

        init_parameters()

    def get_definition(self):
        return self.l_definition

    def get_args(self):
        return self.args

    # Definition of linking functions
    # GraphConv
    def l_graph_conv(self, edge_features):
        # input is edge_features, send it into learn_modules
        last_layer_output = edge_features
        for layer in self.learn_modules:
            # use initially built neural network
            last_layer_output = layer(last_layer_output)
        return last_layer_output[:, 0, :, :]


    def init_graph_conv(self):
        input_size = self.args['edge_feature_size']
        hidden_size = self.args['link_hidden_size']

        # build the neural network initially
        if self.args.get('link_relu', False):
            self.learn_modules.append(torch.nn.ReLU())
            self.learn_modules.append(torch.nn.Dropout())

        for _ in range(self.args['link_hidden_layers']-1):                         # append n conv operation
            self.learn_modules.append(torch.nn.Conv2d(input_size, hidden_size, 1)) # input channels: input_size, decides channel of filter
                                                                                   # output_channels: output_size, decides number of filter used
                                                                                   # kernel_size: decides filter size used
            self.learn_modules.append(torch.nn.ReLU())
            # self.learn_modules.append(torch.nn.Dropout())
            # self.learn_modules.append(torch.nn.BatchNorm2d(hidden_size))
            input_size = hidden_size

        self.learn_modules.append(torch.nn.Conv2d(input_size, 1, 1))               # map channel to 1, in order to have a matrix
        # self.learn_modules.append(torch.nn.Sigmoid())

In [ ]:
edge_feature_size = edge_features.shape[2]
node_feature_size = node_features.shape[1]
message_size = edge_feature_size

model_args = {'edge_feature_size': edge_feature_size, 'node_feature_size': node_feature_size, 'message_size': message_size, 'link_hidden_size': 512, 'link_hidden_layers': 2, 'link_relu': False, 'update_hidden_layers': 1, 'update_dropout': False, 'update_bias': True, 'propagate_layers': 3, 'hoi_classes': 117, 'resize_feature_to_message_size': False}
link_fun = LinkFunction('GraphConv', model_args)
